In [19]:
import time
from time import sleep

from random import randint

import pandas as pd

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import *

from webdriver_manager.chrome import ChromeDriverManager

In [20]:
# transformando o arquivo excel em dataframe
cartas_df = pd.read_excel('excel/lista_cartas_magic_com_edicao.xlsx')

# eliminando as duplicatas (cartas com o mesmo nome e edições diferentes)
cartas_df = cartas_df.drop_duplicates(
    subset='nome_portugues', keep="first").reset_index(drop=True)

In [21]:
cartas_df

,nome_portugues,nome_ingles,edicao
0,Tutor Vampírico,Vampiric Tutor,Sexta Edição Classica
1,Proteção Oscilante,Flickering Ward,Tempestade
2,Desenterrar,Unearth,O Legado de Urza
3,Aluren,Aluren,Tempestade
4,Tutor Místico,Mystical Tutor,Sexta Edição Classica
5,Abertura Temporal,Temporal Aperture,A Saga de Urza
6,Aranha Escondida,Hidden Spider,A Saga de Urza
7,Guerra Justa,Righteous War,Visões
8,Bênção de Cho-Manno,Cho-Manno's Blessing,Máscara de Mercádia
9,Juramento dos Regentes,Oath of Lieges,Êxodo


In [22]:
# algumas configurações para o webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--proxy-server='direct://'")
chrome_options.add_argument("--proxy-bypass-list=*")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--ignore-certificate-errors')

# instanciando a versão do webdriver que será instalada na máquina local
service = Service(ChromeDriverManager().install())

# instanciando o webdriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Extração

In [23]:
nome_portugues_lista = []
nome_ingles_lista = []
edicao_lista = []
artista_lista = []
raridade_lista = []
valor_medio_srt_lista = []

cartas_web_dic = []

# efetuando busca no site
for portugues, ingles in zip(cartas_df['nome_portugues'], cartas_df['nome_ingles']):

    driver.get(f"https://www.ligamagic.com/?view=cards/card&card={portugues}&aux={ingles}")

    sleep(randint(2, 4))

    action = ActionChains(driver)

    # selecionar a edição
    edcard = 0

    while True:
        try:
            edicao_menu = driver.find_element(
                By.CSS_SELECTOR, f"#edcard_{edcard}> img:nth-child(1)")
            action.move_to_element(edicao_menu).click().perform()

        except NoSuchElementException:
            break
        else:
            edcard += 1

        # pegar o valor médio da carta
        nome_portugues = portugues
        nome_ingles = ingles
        edicao = driver.find_element(By.XPATH, '//*[@id="ed-nome"]/a').text
        artista = driver.find_element(By.XPATH, '//*[@id="ed-artista"]/a').text
        raridade = driver.find_element(
            By.XPATH, '//*[@id="ed-raridade"]/a').text
        valor_medio_srt = driver.find_element(
            By.XPATH, '//*[@id="card-info"]/div[5]/div[2]/div/div[4]').text

        # exibir nome e edição
        print(f"{edcard}. {nome_portugues} | {nome_ingles} | {edicao} | {artista} | {raridade} | {valor_medio_srt}")
        
        # criação do dicionário
        cartas_web_dic.append({
            'nome_portugues': nome_portugues,
            'nome_ingles': nome_ingles,
            'edicao': edicao,
            'artista': artista,
            'raridade': raridade,
            'valor_medio_srt': valor_medio_srt
        })

1. Tutor Vampírico | Vampiric Tutor  | Commander Legends (Variantes) | Raymond Swanland | Mítica | R$ 420,78
2. Tutor Vampírico | Vampiric Tutor  | Commander Legends | Raymond Swanland | Mítica | R$ 284,96
3. Tutor Vampírico | Vampiric Tutor  | The List | Raymond Swanland | Mítica | R$ 281,48
4. Tutor Vampírico | Vampiric Tutor  | Eternal Masters | Raymond Swanland | Mítica | R$ 277,59
5. Tutor Vampírico | Vampiric Tutor  | Sexta Edição Classica | Gary Leach | Rara | R$ 343,63
6. Tutor Vampírico | Vampiric Tutor  | Visões | Gary Leach | Rara | R$ 302,35
7. Tutor Vampírico | Vampiric Tutor  | Vintage Masters | Gary Leach | Rara | R$ 149,99
8. Tutor Vampírico | Vampiric Tutor  | Judge Gift Program | Gary Leach | Special | R$ 1.595,95
9. Tutor Vampírico | Vampiric Tutor  | World Championship Decks | Gary Leach | Special | R$ 180,00
1. Proteção Oscilante | Flickering Ward  | Tempestade | Greg Simanson | Incomum | R$ 59,35
1. Desenterrar | Unearth | Double Masters 2022 (Borderless) | Jeff M

In [24]:
# corversão do dicionário para dataframe
cartas_web_df = pd.DataFrame().from_dict(cartas_web_dic)

cartas_web_df

,nome_portugues,nome_ingles,edicao,artista,raridade,valor_medio_srt
0,Tutor Vampírico,Vampiric Tutor,Commander Legends (Variantes),Raymond Swanland,Mítica,"R$ 420,78"
1,Tutor Vampírico,Vampiric Tutor,Commander Legends,Raymond Swanland,Mítica,"R$ 284,96"
2,Tutor Vampírico,Vampiric Tutor,The List,Raymond Swanland,Mítica,"R$ 281,48"
3,Tutor Vampírico,Vampiric Tutor,Eternal Masters,Raymond Swanland,Mítica,"R$ 277,59"
4,Tutor Vampírico,Vampiric Tutor,Sexta Edição Classica,Gary Leach,Rara,"R$ 343,63"
5,Tutor Vampírico,Vampiric Tutor,Visões,Gary Leach,Rara,"R$ 302,35"
6,Tutor Vampírico,Vampiric Tutor,Vintage Masters,Gary Leach,Rara,"R$ 149,99"
7,Tutor Vampírico,Vampiric Tutor,Judge Gift Program,Gary Leach,Special,"R$ 1.595,95"
8,Tutor Vampírico,Vampiric Tutor,World Championship Decks,Gary Leach,Special,"R$ 180,00"
9,Proteção Oscilante,Flickering Ward,Tempestade,Greg Simanson,Incomum,"R$ 59,35"


In [25]:
# fechando o site
time.sleep(10)
driver.close()

# Transformação

In [26]:
# transformando o valor médio string para float
cartas_web_df['valor_medio'] = list(map(lambda x : x.split(" ")[1].replace(".", "").replace(",", "."), cartas_web_df['valor_medio_srt']))
cartas_web_df['valor_medio'] = list(map(lambda x : float(x), cartas_web_df['valor_medio']))

cartas_web_df

,nome_portugues,nome_ingles,edicao,artista,raridade,valor_medio_srt,valor_medio
0,Tutor Vampírico,Vampiric Tutor,Commander Legends (Variantes),Raymond Swanland,Mítica,"R$ 420,78",420.78
1,Tutor Vampírico,Vampiric Tutor,Commander Legends,Raymond Swanland,Mítica,"R$ 284,96",284.96
2,Tutor Vampírico,Vampiric Tutor,The List,Raymond Swanland,Mítica,"R$ 281,48",281.48
3,Tutor Vampírico,Vampiric Tutor,Eternal Masters,Raymond Swanland,Mítica,"R$ 277,59",277.59
4,Tutor Vampírico,Vampiric Tutor,Sexta Edição Classica,Gary Leach,Rara,"R$ 343,63",343.63
5,Tutor Vampírico,Vampiric Tutor,Visões,Gary Leach,Rara,"R$ 302,35",302.35
6,Tutor Vampírico,Vampiric Tutor,Vintage Masters,Gary Leach,Rara,"R$ 149,99",149.99
7,Tutor Vampírico,Vampiric Tutor,Judge Gift Program,Gary Leach,Special,"R$ 1.595,95",1595.95
8,Tutor Vampírico,Vampiric Tutor,World Championship Decks,Gary Leach,Special,"R$ 180,00",180.00
9,Proteção Oscilante,Flickering Ward,Tempestade,Greg Simanson,Incomum,"R$ 59,35",59.35


In [27]:
# calculando a comissão do mercado livre

def comissao_ml(valor_medio) -> int:
    ''' calculo de porcentagem (ml) e valor final do produto:
        - valores maiores ou iguais a R$ 79,00 a comissão do ML é de 11,5%
        - valores menores que R$ 79,00 a comissão do ML é de 11,5% + R$ 5,00
    '''
    if valor_medio >= 79:
        porcentagem_ml = (0.115 * valor_medio)
    else:
        porcentagem_ml = (0.115 * valor_medio) + 5

    valor_ml = round((porcentagem_ml + valor_medio), 2)
    
    return valor_ml


cartas_web_df['valor_ml'] = list(map(lambda x: comissao_ml(x), cartas_web_df['valor_medio']))

cartas_web_df

,nome_portugues,nome_ingles,edicao,artista,raridade,valor_medio_srt,valor_medio,valor_ml
0,Tutor Vampírico,Vampiric Tutor,Commander Legends (Variantes),Raymond Swanland,Mítica,"R$ 420,78",420.78,469.17
1,Tutor Vampírico,Vampiric Tutor,Commander Legends,Raymond Swanland,Mítica,"R$ 284,96",284.96,317.73
2,Tutor Vampírico,Vampiric Tutor,The List,Raymond Swanland,Mítica,"R$ 281,48",281.48,313.85
3,Tutor Vampírico,Vampiric Tutor,Eternal Masters,Raymond Swanland,Mítica,"R$ 277,59",277.59,309.51
4,Tutor Vampírico,Vampiric Tutor,Sexta Edição Classica,Gary Leach,Rara,"R$ 343,63",343.63,383.15
5,Tutor Vampírico,Vampiric Tutor,Visões,Gary Leach,Rara,"R$ 302,35",302.35,337.12
6,Tutor Vampírico,Vampiric Tutor,Vintage Masters,Gary Leach,Rara,"R$ 149,99",149.99,167.24
7,Tutor Vampírico,Vampiric Tutor,Judge Gift Program,Gary Leach,Special,"R$ 1.595,95",1595.95,1779.48
8,Tutor Vampírico,Vampiric Tutor,World Championship Decks,Gary Leach,Special,"R$ 180,00",180.00,200.70
9,Proteção Oscilante,Flickering Ward,Tempestade,Greg Simanson,Incomum,"R$ 59,35",59.35,71.18


In [28]:
# interseção das duas tabelas (para retornar apenas as edições de interece)
cartas_final_df = pd.merge(cartas_df, cartas_web_df, how='left', on=(
    'nome_portugues', 'nome_ingles', 'edicao'))

cartas_final_df

,nome_portugues,nome_ingles,edicao,artista,raridade,valor_medio_srt,valor_medio,valor_ml
0,Tutor Vampírico,Vampiric Tutor,Sexta Edição Classica,Gary Leach,Rara,"R$ 343,63",343.63,383.15
1,Proteção Oscilante,Flickering Ward,Tempestade,Greg Simanson,Incomum,"R$ 59,35",59.35,71.18
2,Desenterrar,Unearth,O Legado de Urza,Don Hazeltine,Comum,"R$ 4,89",4.89,10.45
3,Aluren,Aluren,Tempestade,April Lee,Rara,"R$ 358,01",358.01,399.18
4,Tutor Místico,Mystical Tutor,Sexta Edição Classica,David O`Connor,Incomum,"R$ 87,52",87.52,97.58
5,Abertura Temporal,Temporal Aperture,A Saga de Urza,Michael Sutfin,Rara,"R$ 52,34",52.34,63.36
6,Aranha Escondida,Hidden Spider,A Saga de Urza,Thomas M. Baxa,Comum,"R$ 3,89",3.89,9.34
7,Guerra Justa,Righteous War,Visões,Ian Miller,Rara,"R$ 26,59",26.59,34.65
8,Bênção de Cho-Manno,Cho-Manno's Blessing,Máscara de Mercádia,John Matson,Comum,"R$ 4,05",4.05,9.52
9,Juramento dos Regentes,Oath of Lieges,Êxodo,Mark Zug,Rara,"R$ 26,83",26.83,34.92


In [29]:
# teste para saber se foi realmente retoranda a lista desejada
if len(cartas_df["nome_portugues"]) == len(cartas_final_df["nome_portugues"]):
    print('Ok!')
else:
    print('Deu errado!')

Ok!


# Carregamento

In [30]:
# geração do arquivo final
cartas_final_df.to_excel("excel/cartas_magic_output.xlsx")
cartas_final_df

,nome_portugues,nome_ingles,edicao,artista,raridade,valor_medio_srt,valor_medio,valor_ml
0,Tutor Vampírico,Vampiric Tutor,Sexta Edição Classica,Gary Leach,Rara,"R$ 343,63",343.63,383.15
1,Proteção Oscilante,Flickering Ward,Tempestade,Greg Simanson,Incomum,"R$ 59,35",59.35,71.18
2,Desenterrar,Unearth,O Legado de Urza,Don Hazeltine,Comum,"R$ 4,89",4.89,10.45
3,Aluren,Aluren,Tempestade,April Lee,Rara,"R$ 358,01",358.01,399.18
4,Tutor Místico,Mystical Tutor,Sexta Edição Classica,David O`Connor,Incomum,"R$ 87,52",87.52,97.58
5,Abertura Temporal,Temporal Aperture,A Saga de Urza,Michael Sutfin,Rara,"R$ 52,34",52.34,63.36
6,Aranha Escondida,Hidden Spider,A Saga de Urza,Thomas M. Baxa,Comum,"R$ 3,89",3.89,9.34
7,Guerra Justa,Righteous War,Visões,Ian Miller,Rara,"R$ 26,59",26.59,34.65
8,Bênção de Cho-Manno,Cho-Manno's Blessing,Máscara de Mercádia,John Matson,Comum,"R$ 4,05",4.05,9.52
9,Juramento dos Regentes,Oath of Lieges,Êxodo,Mark Zug,Rara,"R$ 26,83",26.83,34.92
